## Start Node

In [1]:
import rclpy
from nimbro_utils.lazy import start_and_spin_node, stop_node

In [2]:
class MyNode(rclpy.node.Node):
    def __init__(self, context=None):
        super().__init__("test_tf_oracle_node", context=context)

In [3]:
node, executor, context, thread = start_and_spin_node(MyNode, blocking=False)

> Starting node 'MyNode'


## Add TfOracle

In [4]:
from nimbro_utils.lazy import TfOracle

In [5]:
node.tf_oracle = TfOracle(node)

#### The TfOracle's settings can be obtained via `get_settings()` and updated via `set_settings()`.

In [6]:
node.tf_oracle.get_settings()

{'severity': 20,
 'suffix': 'tf_oracle',
 'tf_buffer': 'tf_buffer',
 'tf_listener': 'tf_listener'}

In [7]:
node.tf_oracle.set_settings(settings={'severity': 20})

## Get transform

In [8]:
success, message, transform = node.tf_oracle.get_transform(
    source_frame="gemini_color_optical_frame",
    target_frame="gemini_link",
    target_format="TransformStamped", # default
    mute_lookup_error=False, # default
    time=None, # default
    max_age=None, # default
    timeout=1.0 # default
)

In [9]:
print(success)
print(message)
print(transform)

True
Looked up '1757522721.659s' old transform from 'gemini_color_optical_frame' to 'gemini_link' in format 'TransformStamped' after waiting '0.000s'.
geometry_msgs.msg.TransformStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id='gemini_link'), child_frame_id='gemini_color_optical_frame', transform=geometry_msgs.msg.Transform(translation=geometry_msgs.msg.Vector3(x=0.001982434391975403, y=-0.014101850509643554, z=-8.649254590272904e-05), rotation=geometry_msgs.msg.Quaternion(x=-0.49786300261868527, y=0.5014592046254752, z=-0.4984931680273193, w=0.5021709451471476)))


## Obtain frames as PointStamped

In [10]:
success, message, point_color = node.tf_oracle.get_transform(
    source_frame="gemini_color_optical_frame",
    target_frame="gemini_link",
    target_format="PointStamped",
    mute_lookup_error=False, # default
    time=None, # default
    max_age=None, # default
    timeout=1.0 # default
)

In [11]:
print(success)
print(message)
print(point_color)

True
Looked up '1757522721.677s' old transform from 'gemini_color_optical_frame' to 'gemini_link' in format 'PointStamped' after waiting '0.000s'.
geometry_msgs.msg.PointStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id='gemini_link'), point=geometry_msgs.msg.Point(x=0.001982434391975403, y=-0.014101850509643554, z=-8.649254590272904e-05))


In [12]:
success, message, point_depth = node.tf_oracle.get_transform(
    source_frame="gemini_depth_optical_frame",
    target_frame="gemini_link",
    target_format="PointStamped",
    mute_lookup_error=False, # default
    time=None, # default
    max_age=None, # default
    timeout=1.0 # default
)

In [13]:
print(success)
print(message)
print(point_depth)

True
Looked up '1757522721.699s' old transform from 'gemini_depth_optical_frame' to 'gemini_link' in format 'PointStamped' after waiting '0.000s'.
geometry_msgs.msg.PointStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id='gemini_link'), point=geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0))


## Transorm to different frame

In [14]:
success, message, point_depth_transformed = node.tf_oracle.transform_to_frame(
    point_or_pose_or_cloud=point_depth,
    target_frame="gemini_depth_optical_frame"
)

In [15]:
print(success)
print(message)
print(point_depth_transformed)

True
Transformed provided geometry message from frame 'gemini_link' to frame 'gemini_depth_optical_frame'.
geometry_msgs.msg.PointStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id='gemini_depth_optical_frame'), point=geometry_msgs.msg.Point(x=0.0, y=0.0, z=0.0))


## Measure distance between frames

In [16]:
success, message, distance = node.tf_oracle.get_distance(
    point_or_pose_a=point_color,
    point_or_pose_b=point_depth,
    # point_or_pose_a="gemini_color_optical_frame",
    # point_or_pose_b="gemini_depth_optical_frame",
)

In [17]:
print(success)
print(message)
print(distance)

True
The distance between the two points is '0.014m'.
0.014240776484283355


## Interpolate between frames

In [18]:
success, message, interpolated_point = node.tf_oracle.get_interpolation(
    # point_or_pose_a=point_color,
    # point_or_pose_b=point_depth,
    point_or_pose_a="gemini_color_optical_frame",
    point_or_pose_b="gemini_depth_optical_frame",
    mode='mix', # default
    x=0.5 # default
)

In [19]:
print(success)
print(message)
print(interpolated_point)

True
Successfully interpolated between two points using mode 'mix' with parameter '0.5'.
geometry_msgs.msg.PointStamped(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id='gemini_color_optical_frame'), point=geometry_msgs.msg.Point(x=-0.007051138960114438, y=6.71341127336713e-06, z=-0.0009906174913127746))


## Stop Node

In [20]:
stop_node(node, executor, context, thread)

> Stopped node 'MyNode'
